In [1]:
from question_generation import *

In [2]:
sub_questions = []
sub_questions = generate_sub_questions("What is the company's growth strategy?")

In [3]:
sub_questions

['What specific markets or segments is the company targeting for growth in the upcoming years?',
 "What are the key initiatives or projects outlined in the company's 10-K that support its growth strategy?",
 'How does the company plan to leverage its existing resources or capabilities to drive growth?',
 "What role do acquisitions or partnerships play in the company's growth strategy as mentioned in the SEC documents?",
 "What financial projections or goals related to growth are provided in the company's 10-K filing?",
 'How does the company assess and manage risks associated with its growth strategy according to its SEC disclosures?']

In [5]:
from data_preprocessing import *

with open('data/10Ks/axp-2024-10k.pdf', "rb") as file:
    # Preprocess the document
    raw_text = preprocess_data(file)

In [6]:
len(raw_text)

821074